In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv']


In [2]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [3]:
print(train_df.shape)
print(test_df.shape)

(15035, 21)
(6468, 20)


## 변수 생성

In [4]:
#create column for new variable (one is enough because it's a binary categorical feature)
#if area>0 it gets 1, for area==0 it gets 0 
train_df['HasBsmt'] = pd.Series(len(train_df['sqft_basement']), index=train_df.index)
train_df['HasBsmt'] = 0
train_df.loc[train_df['sqft_basement']>0,'HasBsmt'] = 1

In [5]:
#create column for new variable (one is enough because it's a binary categorical feature)
#if area>0 it gets 1, for area==0 it gets 0 
test_df['HasBsmt'] = pd.Series(len(test_df['sqft_basement']), index=test_df.index)
test_df['HasBsmt'] = 0 
test_df.loc[test_df['sqft_basement']>0,'HasBsmt'] = 1

In [6]:
#transform data
train_df.loc[train_df['HasBsmt']==1,'sqft_basement'] = np.log(train_df['sqft_basement'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [7]:
#transform data
test_df.loc[test_df['HasBsmt']==1,'sqft_basement'] = np.log(test_df['sqft_basement'])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [8]:
train_df = pd.get_dummies(train_df.loc[:,train_df.columns != 'date'])
test_df = pd.get_dummies(test_df.loc[:,test_df.columns != 'date'])

In [9]:
train_df['price'] = np.log(train_df['price'])

In [10]:
train_y = train_df['price']
train_X = train_df.loc[:, train_df.columns != 'price']
train_X = train_X.loc[:, train_X.columns != 'id']

In [11]:
print(train_X.columns)
print(len(train_X.columns))
print(test_df.columns)
print(len(test_df.columns))

Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'HasBsmt'],
      dtype='object')
19
Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'HasBsmt'],
      dtype='object')
20


In [12]:
print('train_X shape : ', train_X.shape)
print('test_df shape : ', test_df.shape)

train_X shape :  (15035, 19)
test_df shape :  (6468, 20)


## validation set split

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.1)

In [15]:
print(train_X.shape)
print(val_X.shape)
print(train_y.shape)
print(val_y.shape)

(13531, 19)
(1504, 19)
(13531,)
(1504,)


## Catboost import

In [16]:
from catboost import CatBoostRegressor

In [17]:
cat_model = CatBoostRegressor(iterations=250, learning_rate=0.05, depth=5)
cat_model.fit(train_X, train_y)

0:	learn: 12.4092949	total: 64ms	remaining: 15.9s
1:	learn: 11.7926020	total: 76.2ms	remaining: 9.45s
2:	learn: 11.2052265	total: 87.4ms	remaining: 7.2s
3:	learn: 10.6478837	total: 98.2ms	remaining: 6.04s
4:	learn: 10.1187549	total: 109ms	remaining: 5.32s
5:	learn: 9.6154863	total: 119ms	remaining: 4.82s
6:	learn: 9.1371598	total: 130ms	remaining: 4.5s
7:	learn: 8.6827174	total: 140ms	remaining: 4.24s
8:	learn: 8.2502474	total: 147ms	remaining: 3.93s
9:	learn: 7.8403666	total: 158ms	remaining: 3.8s
10:	learn: 7.4502715	total: 169ms	remaining: 3.67s
11:	learn: 7.0807935	total: 181ms	remaining: 3.58s
12:	learn: 6.7291282	total: 191ms	remaining: 3.49s
13:	learn: 6.3954694	total: 202ms	remaining: 3.4s
14:	learn: 6.0777893	total: 212ms	remaining: 3.32s
15:	learn: 5.7758131	total: 220ms	remaining: 3.22s
16:	learn: 5.4892731	total: 231ms	remaining: 3.16s
17:	learn: 5.2169890	total: 242ms	remaining: 3.12s
18:	learn: 4.9587193	total: 253ms	remaining: 3.08s
19:	learn: 4.7138224	total: 266ms	rema

In [18]:
val_y = np.exp(val_y)

In [19]:
val_result = cat_model.predict(val_X)
val_result = np.exp(val_result)
RMSE = ((np.sum(val_result - val_y) ** 2) / val_result.shape[0] ) ** 0.5
print(RMSE)

846589.4517858367


In [20]:
result = cat_model.predict(test_df.loc[:, test_df.columns != 'id' ])

In [21]:
result = np.exp(result)

In [22]:
submission = pd.DataFrame({'id': test_df['id'],
                           'price': result})

submission.to_csv('./submission.csv', index=False)